## **MIDI Factory**

#### Import Libraries

In [507]:
import pandas as pd
import xlwings as xw
import math

from miditoolkit.midi import parser as mid_parser  
from miditoolkit.midi import containers as ct

### load midi file

In [508]:
path_midi = 'test.mid'
midi_obj = mid_parser.MidiFile(path_midi)

### Constant

In [509]:
ticks_per_beat = round(midi_obj.ticks_per_beat / 10)
time_signature = midi_obj.time_signature_changes[0]
numerator =  time_signature.numerator
_denominator =  time_signature.denominator
tempo = round(midi_obj.tempo_changes[0].tempo)
restNote = None

rhythms = {
    round(ticks_per_beat * 4): "w", 
    round(ticks_per_beat * 3): "hd",
    round(ticks_per_beat * 4 / 3): "ht",
    round(ticks_per_beat * 2): "h",
    round(ticks_per_beat * 1.5): "qd",
    round(ticks_per_beat * 2 / 3): "qt",
    round(ticks_per_beat * 1 + ticks_per_beat * 0.25): "q^s",
    ticks_per_beat: "q",
    round(ticks_per_beat * 1 / 3):"et", 
    round(ticks_per_beat * 3 / 4): "ed",
    round(ticks_per_beat / 2): "e",
    round(ticks_per_beat / 4 ): "s",
    round(ticks_per_beat / 6): "st"
}

theme_start = "theme_start"
phrase_start = "phrase_start"

bar_duration = numerator * ticks_per_beat

print('✥ ticks per beat:', ticks_per_beat)
print('✥ bar duration:', bar_duration)
print('✥ tempo:', tempo, "BPM")

✥ ticks per beat: 96
✥ bar duration: 384
✥ tempo: 200 BPM


### Define RHYTHMS

In [510]:
def rhythms_define(duration: int):
    return duration
    for i in range(-2, 3):
        rhythm = rhythms.get(duration + i)
        if rhythm:
            return rhythm

### Get Markers

In [511]:
markers = {}
markers_set = set()

print('Markers dictionary :-')
for marker in midi_obj.markers:
    marker_titles = marker.text.split(',')
    markers[round(marker.time / 10)] = marker_titles
    for m in marker_titles:
        markers_set.add(m)
    print(f'  ✥ {round(marker.time / 10)} : {marker_titles}')

Markers dictionary :-
  ✥ 384 : ['theme_start', 'trill']
  ✥ 1152 : ['phrase_start']
  ✥ 2208 : ['Gliss']
  ✥ 3840 : ['theme_start']
  ✥ 6816 : ['Portato']
  ✥ 10752 : ['theme_start']
  ✥ 12288 : ['Vibrato']


### Get Pitch bend

In [512]:
pitch_bends_raw = midi_obj.instruments[0].pitch_bends

pitch_bends_filtered = list(filter(lambda item: item.pitch != 0, pitch_bends_raw))

pitch_bends = {}

print('Pitch bend :-')
for pitch_bend in pitch_bends_filtered:
    pitch_bends[round(pitch_bend.time / 10)] = pitch_bend.pitch
    print(f'  ✥ {round(pitch_bend.time / 10)} : {pitch_bend.pitch}')

Pitch bend :-
  ✥ 1200 : -341


#### Create empty Series

In [513]:
empty_series = pd.Series()

### Create DataFrame with empty Series

In [514]:
midi_df = pd.DataFrame({
    'note': empty_series,
    'interval': empty_series,
    'note_start_time': empty_series,
    'duration': empty_series,
    'slur': empty_series,
    'Velocity': empty_series,
})

for column in markers_set:
    midi_df[column] = empty_series

### Load your MIDI file

In [515]:
# Initialize a dictionary to store note start times and velocities
note_start_times = {}
note_velocities = {}
techniques = {}
interval_for_new_melody_begin = None
new_melody_beginning_marker = "theme_start"
new_phrase_beginning_marker = "phrase_start"

last_note_end_time = 0
quarter_tone = False
last_note = None

notes = midi_obj.instruments[0].notes

for note in notes:
    pitch = note.pitch
    start_time = round(note.start / 10)
    end_time = round(note.end / 10)
    pitch_bend = pitch_bends.get(start_time)
    quarter_tone = pitch_bend / 341 * 0.5 if(pitch_bend) else 0
    rest_exist = start_time - last_note_end_time > 0
    
    # handle rest if exist
    if (rest_exist):
        rest_start_time = last_note_end_time
        next_note_start_time = start_time
        
        rest_starts_bar = math.ceil((rest_start_time + 1) / bar_duration)
        rest_ends_bar = math.ceil((next_note_start_time - 1) / bar_duration)

        existed_rest_bars = rest_ends_bar - rest_starts_bar
        
        for i in range(existed_rest_bars):
            bar_ends = (rest_starts_bar + i) * bar_duration
            rest_row_temp = {key: 0 for key in midi_df.columns}
            rest_row_temp['note'] = restNote
            rest_row_temp['note_start_time'] = rest_start_time
            rest_row_temp['duration'] = rhythms_define(bar_ends - rest_start_time)
            rest_row_temp['Velocity'] = None
                    
            markers_list = markers.get(rest_start_time)
            if(markers_list):
                for m_key in markers_list:
                    rest_row_temp[m_key] = 1
            
            # Add a new rest event to the DataFrame  
            midi_df.loc[len(midi_df)] = rest_row_temp
            
            # update new rest start point
            rest_start_time = bar_ends
            
        # existed_rest_less_than_bar 
        if(next_note_start_time - rest_start_time):
            rest_row_temp = {key: 0 for key in midi_df.columns}
            rest_row_temp['note'] = restNote
            rest_row_temp['note_start_time'] = rest_start_time
            rest_row_temp['duration'] = rhythms_define(start_time - rest_start_time)
            rest_row_temp['Velocity'] = None
            
            markers_list = markers.get(rest_start_time)
            if(markers_list):
                for m_key in markers_list:
                    rest_row_temp[m_key] = 1

            # Add a new rest event to the DataFrame  
            midi_df.loc[len(midi_df)] = rest_row_temp
    
    note_starts_bar = math.ceil((start_time + 1) / bar_duration)
    note_ends_bar = math.ceil((end_time - 1) / bar_duration)
    nested_slur_note = False

    existed_slur_note = note_ends_bar - note_starts_bar
    if(existed_slur_note):
        for i in range(existed_slur_note):
            bar_ends = (note_starts_bar + i) * bar_duration
            slur_note_row_temp = {key: 0 for key in midi_df.columns}
            slur_note_row_temp['note'] = pitch + quarter_tone
            slur_note_row_temp['note_start_time'] = start_time
            slur_note_row_temp['duration'] = rhythms_define(bar_ends - start_time)
            slur_note_row_temp['Velocity'] = note.velocity
            
            if(nested_slur_note):
                slur_note_row_temp['slur'] = 1
            else:
                nested_slur_note = not nested_slur_note
            
            markers_list = markers.get(start_time)
            if(markers_list):
                for m_key in markers_list:
                    slur_note_row_temp[m_key] = 1
            
            # handle intervals
            if(slur_note_row_temp.get(theme_start) == 0 and slur_note_row_temp.get(phrase_start) == 0):
                last_note_int = last_note or 0
                current_note = slur_note_row_temp['note'] or 0
                interval = current_note - last_note_int
                slur_note_row_temp['interval'] =  interval if(start_time - last_note_end_time < bar_duration) else None
            else:
                slur_note_row_temp['interval'] = None
            
            # Add a new rest event to the DataFrame  
            midi_df.loc[len(midi_df)] = slur_note_row_temp
            
            # update new rest start point
            last_note_end_time = bar_ends
            start_time = bar_ends
            last_note = slur_note_row_temp['note']
            
        last_slur_note_row_temp = {key: 0 for key in midi_df.columns}
        last_slur_note_row_temp['note'] = pitch + quarter_tone
        last_slur_note_row_temp['note_start_time'] = start_time
        last_slur_note_row_temp['duration'] = rhythms_define(end_time - start_time)
        last_slur_note_row_temp['Velocity'] = note.velocity
        last_slur_note_row_temp['slur'] = 1
                
        markers_list = markers.get(start_time)
        if(markers_list):
            for m_key in markers_list:
                last_slur_note_row_temp[m_key] = 1
        
        # handle intervals
        if(last_slur_note_row_temp.get(theme_start) == 0 and last_slur_note_row_temp.get(phrase_start) == 0):
            last_note_int = last_note or 0
            current_note = last_slur_note_row_temp['note'] or 0
            interval = current_note - last_note_int
            last_slur_note_row_temp['interval'] =  interval if(start_time - last_note_end_time < bar_duration) else None
        else:
            last_slur_note_row_temp['interval'] = None
        
        # Add a new rest event to the DataFrame  
        midi_df.loc[len(midi_df)] = last_slur_note_row_temp
        
        # update new rest start point
        last_note_end_time = end_time
        last_note = last_slur_note_row_temp['note']
    else:
        # handel Note
        note_row_temp = {key: 0 for key in midi_df.columns}
        note_row_temp['note'] = pitch + quarter_tone
        note_row_temp['note_start_time'] = start_time
        note_row_temp['duration'] = rhythms_define(end_time - start_time)
        note_row_temp['Velocity'] = note.velocity
        
        
        markers_list = markers.get(start_time)
        if(markers_list):
            for m_key in markers_list:
                note_row_temp[m_key] = 1
        
        
        # handle intervals
        if(note_row_temp.get(theme_start) == 0 and note_row_temp.get(phrase_start) == 0):
            last_note_int = last_note or 0
            current_note = note_row_temp['note'] or 0
            interval = current_note - last_note_int
            note_row_temp['interval'] =  interval if(start_time - last_note_end_time < bar_duration) else None
        else:
            note_row_temp['interval'] = None
        
        
        # Add a new rest event to the DataFrame
        midi_df.loc[len(midi_df)] = note_row_temp
        
        
        last_note_end_time = end_time
        last_note = note_row_temp['note']

### Update Excel File

In [516]:
# Connect to the workbook
workbook = xw.Book('test.xlsx')

# Select the worksheet
worksheet = workbook.sheets['Sheet1']

# Delete the existing table (if it exists)
if worksheet.tables:
    worksheet.range(worksheet.tables[0].range.address).api.Delete()

# Add a new table to the worksheet
table = worksheet.tables.add(source=worksheet.range('A1'), name='MyTable')

# Update the table with the new data
table.update(midi_df)

# Show the updated range of cells in a new window
table.range.api.Select()

True